# 8章 エラーと例外

## 8.1 構文エラー
パース上のエラー。矢印よりも前のトークンが原因。

## 8.2 例外
構文的に正しくてもexceptionエラーが発生することがある。

In [1]:
#ゼロディバイド
10 * (1/0)

ZeroDivisionError: division by zero

In [2]:
#未定義エラー
4 + spam*3

NameError: name 'spam' is not defined

In [3]:
#型の不一致エラー
'2' + 2

TypeError: can only concatenate str (not "int") to str

## 8.3 例外の処理
選択した例外を処理するように記載することが可能
<br>
Crtl + C等で強制終了することもできるがその際はKeyboardInterruptエラーとなる。

In [8]:
#try節がまず処理され、そのあと、例外が検出された際にはexceptが実行される。
while True:
    try:
        d = int(input("数字を入れてください:"))
        break
    except ValueError:
        print("あらら！これは有効な数字ではありません。どうぞもう一度...")

数字を入れてください:v
あらら！これは有効な数字ではありません。どうぞもう一度...
数字を入れてください:1


In [10]:
#単一文で複数の例外を指定することも可能
#例外名を省略することでワイルドカード（すべて指定）とすることができる。
#ただし、ワイルドカード指定をするとプログラミングエラーが検知できなくなるリスクがある。
except(RuntimeError, TypeError, NameError):
    pass
.....

In [11]:
import sys

try:
    f = open('myfile.txt')
    s = f.readline()
    i = int(s.strip())
except OSError as err:
    print("OS error: {0}".format(err))
except ValueError:
    print("データが整数に変換できません。")
except:
    print("予期せぬエラー:", sys.exc_info()[0])
    raise

OS error: [Errno 2] No such file or directory: 'myfile.txt'


In [12]:
#try...exceptにはelse節を入れることができる。
#elseは全てのexceptより後ろでなければならない。tryが例外を送出しなかったときのみ実行される。
#try節にコードを追加するよりもelseを使った方が良い。
for arg in sys.argv[1:]:
    try:
        f = open(arg, 'r')
    except IOError
        print('cannot open', arg)
    else:
        print(arg, 'has', len(f.readlines()), 'lines')
        f.close()

cannot open -f
C:\Users\ktorato\AppData\Roaming\jupyter\runtime\kernel-944952de-a2e2-4475-8adf-a5e60cd84773.json has 12 lines


In [13]:
try:
    raise Exception('spam' , 'eggs')
except Exception as inst:
    print(type(inst)) #例外インスタンスの型 class = Exception
    print(inst.args) #.argsに格納された引数
    print(inst)　　　#引数を直接参照することで、argsにアクセスしなくても同じ結果となる

    x, y = inst.args
    print('x =', x)
    print('y =', y)

<class 'Exception'>
('spam', 'eggs')
('spam', 'eggs')
x = spam
y = eggs


In [14]:
#例外ハンドラーはtry節中で直接発生した例外だけでなく、try節からコールされた関数内での例外も処理する。
def this_fails():
    x = 1/0

try:
    this_fails()
except ZeroDivisionError as err:
    print('ランタイムエラーを処理します：', err)

ランタイムエラーを処理します： division by zero


## 8.4 例外の送出

In [15]:
#raise文では指定の例外を強制的に発生することができる
raise NameError('HiThere')

NameError: HiThere

In [16]:
#raiseに引数を渡さなければ、いったん無視して他の処理を実行する。
try:
    raise NameError('HiThere')
except NameError:
    print('例外が飛んでった')
    raise

例外が飛んでった


NameError: HiThere

## 8.5ユーザー定義例外
例外クラスを作成することで、プログラムは独自の例外を用いることができる。

In [19]:
#例外は直接間接にExceptionクラスから派生したクラスとすべきである。
class MyError(Exception):
    def __init__(self, value):
        self.value = value
    def __str__(self):
        return repr(self.value)

try:
    raise MyError(2*2)
except MyError as e:
    print('My exception occurred, value:', e.value)

My exception occurred, value: 4


In [20]:
#例外クラスには複雑なことはせず、エラー情報をいくつかの属性として提供するにとどめる。
class Error(Exception):
    """このモジュールの例外のベースクラス"""
    pass

class InputError(Error):
    """入力エラーで送出される例外
    
    属性：
      expression -- エラーが起きた入力式
      message -- エラーの説明
    """
    
    def __init__(self, expression, message):
        self.expression = expression
        self.message = message
        
class TransitionError(Error):
    """許可されない状態遷移を起こそうとする操作があれば送出される
    
    属性：
        previous -- 遷移前の状態
        next -- 移ろうとした状態
        message -- その遷移がなぜ許可されないかの説明
    """

def __init__(self, previous, next, message):
    self.previous = previous
    self.next = next
    self.message = message

## 8.6 クリーンアップ動作の定義

In [21]:
#try文にはオプション節があり、クリーナップ動作を定義することを意識したもので、全ての状況で実行される
#finally父子は例外が発生しようがしまいが、try節から出る際に必ず実行される。
try:
    raise KeyboardInterrupt
finally:
    print('Goodbye, world!')

Goodbye, world!


KeyboardInterrupt: 

In [22]:
#break,continue,returnによって抜ける際にもfinallyが出口として使用される。
#外部リソースを使用していた場合、リソースを開放するためにfinally節を使うと良い。
def divide(x, y):
    try:
        result = x / y
    except ZeroDivisionError:
        print("ゼロ除算!")
    else:
        print("答えは", result)
    finally:
        print("finally節実行中")

In [24]:
divide( 2, 1)

答えは 2.0
finally節実行中


In [25]:
divide(2, 0)

ゼロ除算!
finally節実行中


In [26]:
divide("2", "1")

finally節実行中


TypeError: unsupported operand type(s) for /: 'str' and 'str'

In [ ]:
#以下のコードは実行後のしばらくの時間、ファイルを開きっぱなしにしてしまうこと。
for line in open("myfile.txt"):
    print(line, end ="")

In [ ]:
#with文を使うと、ファイルのようなオブジェクトを使用後、クリーンアップしてくれる。
with open("myfile.txt"): as f:
        for line in f:
            print(line, end = "")